In [2]:
from pathlib import Path
from pprint import pprint
from ost import Sentinel1Batch
from ost.helpers import vector, raster
import json
import gdal
import os

# load username and pw to access sentinel files
username = json.load(open('login_data.json'))['username']
pw = json.load(open('login_data.json'))['pw']

# define the project directory
project_dir = Path.home() / "Documents" / "Python" / "sen1ost" / "data"

In [12]:
niedersachsen_fields = json.load(open('data/aoi/niedersachsen.geojson'))

# define the start and end date
start = "2021-12-01"
end = "2021-12-31"

# define product
key = "ASCENDING_VVVH"

In [11]:
target_len = 15

for field_no in range(1,5):
    single_field = niedersachsen_fields['features'][field_no]
    output = {"type":"FeatureCollection","features":[]}

    len_list = sorted([[idx, len(elem[0])] for idx, elem in enumerate(single_field['geometry']['coordinates'])],key=lambda x: x[1],reverse=True)[:target_len]
    reg_len = [i[0] for i in len_list]

    # mutlipolygon to polygon
    for idx, poly in enumerate(single_field['geometry']['coordinates']):
        if len(single_field['geometry']['coordinates'])<target_len or idx in reg_len:
            xfeature = {"type":"Feature", "geometry":{"type":"Polygon"}}
            xfeature['geometry']['coordinates'] = poly
            output['features'].append(xfeature)
            open('data/aoi/tmp_shape.geojson', 'w').write(json.dumps(output,separators=(',',':'),ensure_ascii=False).replace('}},','}},\n'))

    # aoi file name
    aoi_file = 'data/aoi/tmp_shape.geojson'

    # create buffer around aoi
    vector.buffer_shape(aoi_file, str(project_dir) + '/aoi/aoi_tmp.geojson' ,buffer=0.01)
    aoi = vector.aoi_to_wkt(str(project_dir) + '/aoi/aoi_tmp.geojson')

    # initialize the class to s1_grd instance
    s1_grd = Sentinel1Batch(
        project_dir=project_dir,
        aoi=aoi,
        start=start,
        end=end,
        product_type="GRD",
        beam_mode="IW",
        polarisation="*",
        ard_type="OST-RTC", #Radiometrically Terrain Corrected, images are corrected for radiometric distortions along mountainous slopes
        max_workers=4,
    )

    # search for files
    # added scihub username and pw in Project.py line 260
    # if not s1_grd.inventory_file:
        # s1_grd.search()
    s1_grd.search()

    # refine inventory
    s1_grd.refine_inventory()

    # download ascending orbit images
    s1_grd.download(s1_grd.refined_inventory_dict[key], concurrent=10, mirror = '2', uname=username, pword=pw)

    # set ard parameters
    s1_grd.ard_parameters["single_ARD"]["resolution"] = 10  # set output resolution to 100m
    s1_grd.ard_parameters["single_ARD"]["remove_speckle"] = True  # apply a speckle filter
    s1_grd.ard_parameters["single_ARD"]["to_db"] = True

    # preprocess
    s1_grd.grds_to_ards(
        inventory_df=s1_grd.refined_inventory_dict[key],
        timeseries=False,
        timescan=False,
        mosaic=True,
        overwrite=True,
        max_workers=8,
    )

    # convert to tiff and cut to aoi shape
    for u in ['Gamma0_VV_db','Gamma0_VH_db']:
        paths = list(Path(project_dir).rglob(str(u + '.img')))
        for i in paths:
            inpath = str(i)
            outpath = str(project_dir) + '/out/' + single_field['properties']['FLIK'] + '_' + u + '_' + os.path.split(os.path.split(os.path.split(i)[0])[0])[1] + '.tiff'
            ds = gdal.Open(inpath)
            ds = gdal.Translate(outpath, ds)
            ds = None
            raster.mask_by_shape(outpath,outpath, str(project_dir) + '/aoi/tmp_shape.geojson', rescale=False, description=False, to_db=False)

 INFO (12:10:04): Created project directory at /home/david/Documents/Python/sen1ost/data
 INFO (12:10:04): Downloaded data will be stored in: /home/david/Documents/Python/sen1ost/data/download.
 INFO (12:10:04): Inventory files will be stored in: /home/david/Documents/Python/sen1ost/data/inventory.
 INFO (12:10:04): Processed data will be stored in: /home/david/Documents/Python/sen1ost/data/processing.
 INFO (12:10:04): Using /home/david/Documents/Python/sen1ost/data/temp as directory for temporary files.
 INFO (12:10:06): Writing inventory data to geopackage file: /home/david/Documents/Python/sen1ost/data/inventory/full.inventory.gpkg
 INFO (12:10:06): Coverage analysis for DESCENDING tracks in VV VH polarisation.
 INFO (12:10:06): 2 frames for DESCENDING tracks in VV VH polarisation.
 INFO (12:10:06): 2 frames remain after double entry removal
 INFO (12:10:06): Excluding track 66
 INFO (12:10:06): Excluding track 139
 INFO (12:10:06): 1 frames remain after non-AOI overlap
 INFO (12:1

/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/search.py:212: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geo_df = geo_df.append(gdf)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/Project.py:352: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  ) = refine_inventory.search_refinement(
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/refine_inventory.py:246: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_frame = out_frame.append(gdf_date)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the sing

 INFO (12:10:07): Found 1 full coverage mosaics.
 INFO (12:10:07): Coverage analysis for ASCENDING tracks in VV VH polarisation.
 INFO (12:10:07): 2 frames for ASCENDING tracks in VV VH polarisation.
 INFO (12:10:07): 2 frames remain after double entry removal
 INFO (12:10:07): Excluding track 117
 INFO (12:10:07): Excluding track 15
 INFO (12:10:07): 1 frames remain after non-AOI overlap
 INFO (12:10:07): All remaining tracks fully overlap the AOI. Not removing anything.
 INFO (12:10:07):  1 frames remain after removal of non-full AOI crossing
 INFO (12:10:07): Found 1 full coverage mosaics.
--------------------------------------------
 Summing up the info about mosaics
--------------------------------------------

 1 mosaics for mosaic key DESCENDING_VVVH

 1 mosaics for mosaic key ASCENDING_VVVH
 INFO (12:10:07): All scenes are ready for being processed.
 INFO (12:10:07): Deleting processing folder to start from scratch


/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/Project.py:352: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  ) = refine_inventory.search_refinement(
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/refine_inventory.py:246: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_frame = out_frame.append(gdf_date)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/refine_inventory.py:352: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

 INFO (12:10:07): Processing acquisition from 20200502 over track 117.
Executing processing graph
15%30%45%60%.70%.90% done.
Executing operator...
20%60%100%140% done.
Writing...
.13%.60%100% done.
Executing processing graph
18%36%54%.72%90% done.
Executing operator...
20%60%100%140% done.
Writing...
.13%.60%100% done.
Executing operator...
20%100%100%180% done.
Writing...
.13%.60%100% done.
Executing operator...
20%60%100%140% done.
Writing...
.13%.60%100% done.
Executing operator...
20%60%100%180% done.
Writing...
.13%.60%100% done.
 INFO (12:10:37): Created project directory at /home/david/Documents/Python/sen1ost/data
 INFO (12:10:37): Downloaded data will be stored in: /home/david/Documents/Python/sen1ost/data/download.
 INFO (12:10:37): Inventory files will be stored in: /home/david/Documents/Python/sen1ost/data/inventory.
 INFO (12:10:37): Processed data will be stored in: /home/david/Documents/Python/sen1ost/data/processing.
 INFO (12:10:37): Using /home/david/Documents/Python/

...

/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/refine_inventory.py:246: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_frame = out_frame.append(gdf_date)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/refine_inventory.py:352: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_frame = out_frame.append(gdf)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-nu

 INFO (12:14:03): Downloading scene to: /home/david/Documents/Python/sen1ost/data/download/SAR/GRD/2020/05/03/S1B_IW_GRDH_1SDV_20200503T165941_20200503T170006_021420_028AA0_6F41.zip


 INFO: Downloading : 1.01GB [01:02, 16.1MB/s]                             


 INFO (12:15:16): S1B_IW_GRDH_1SDV_20200503T165941_20200503T170006_021420_028AA0_6F41.zip passed the zip test.
 INFO (12:15:16): All products are downloaded.
 INFO (12:15:16): Deleting processing folder to start from scratch
 INFO (12:15:17): Processing acquisition from 20200503 over track 44.
Executing processing graph
15%30%45%60%.70%.90% done.
Executing operator...
20%60%100%140% done.
Writing...
.13%.60%100% done.
Executing processing graph
18%36%54%.72%90% done.
Executing operator...
20%60%100%140% done.
Writing...
.13%.60%100% done.
Executing operator...
20%100%100%180% done.
Writing...
.13%.60%100% done.
Executing operator...
20%60%100%140% done.
Writing...
.13%.60%100% done.
Executing operator...
20%60%100%180% done.
Writing...
.13%.60%100% done.


/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/grd_batch.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/grd_batch.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processing_df = processing_df.append(temp_df)
